In [1]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")

In [1]:
%matplotlib inline
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt


import modelprop



### Test ModelProp 

In [248]:

from argparse import Namespace
args = Namespace(schema='SARA_v1.0',assetcategory='buildings',
 losscategory='structural',taxonomies='["RCPC1"]')

model = modelprop.Main(args)
model.run()

0

In [250]:
model.metadata

{'id': 'SARA_v1.0',
 'assetCategory': 'buildings',
 'lossCategory': 'structural',
 'taxonomy_source': 'GEM',
 'damage_states': 'HAZUS',
 'format': 'continuous',
 'shape': 'logncdf',
 'description': 'GEM-SARA Model, project RIESGOS',
 'taxonomies': ['URM1',
  'URM2',
  'CM',
  'RM',
  'RC1',
  'RC2',
  'RC3',
  'RC4',
  'RCPC1',
  'RCPC464',
  'RCPC2',
  'RCPC111',
  'ADO',
  'WOOD1',
  'WOOD2',
  'STEEL'],
 'limit_states': ['D1', 'D2', 'D3', 'D4', 'D5'],
 'no_damage': 'D0'}